# Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import pathlib
import sys

ROOT = pathlib.Path().resolve().parent

sys.path.insert(0, str(ROOT))

df = pd.read_csv(ROOT / 'data/dataset.csv', sep='\t')
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

## Transformers

Есть пример похожей работы на transformers [ссылка](https://huggingface.co/shahp7575/gpt2-horoscopes)


**Для работы модели текстовой генерации необходимо несколько вещей:**

1. Предобученная модель на массиве текстов
2. Адекватный векторный промт - то, что модель будет продолжать

В качестве фичеров для такой маски мы предлагаем использовать следующий подход:

`<Категория> {Категория} <День недели>`


In [2]:
df['date_fmt'] = df['date'].dt.strftime("%m-%d")
df

,sing_ru,date,month,day,weekday,weekday_ru,text,date_fmt
0,овен,2004-01-01,1,1,3,Четверг,"Любые разногласия во мнениях скоро улягутся, а...",01-01
1,телец,2004-01-01,1,1,3,Четверг,Первый день нового года прекрасно подходит для...,01-01
2,близнецы,2004-01-01,1,1,3,Четверг,Этот день как нельзя лучше подходит для улучше...,01-01
3,рак,2004-01-01,1,1,3,Четверг,"Первый день нового года, да еще понедельник бу...",01-01
4,лев,2004-01-01,1,1,3,Четверг,Львы не сорвутся в начале года с места в карье...,01-01
...,...,...,...,...,...,...,...,...
79291,скорпион,2023-01-01,1,1,6,Воскресенье,Сегодня Скорпионов поглотит новогодняя суета –...,01-01
79292,стрелец,2023-01-01,1,1,6,Воскресенье,"Сегодня у Стрельцов не все идет по плану, и, в...",01-01
79293,козерог,2023-01-01,1,1,6,Воскресенье,Сегодня Козерогам стоит быть во всем осторожне...,01-01
79294,водолей,2023-01-01,1,1,6,Воскресенье,Сегодня Водолеям придется подстраиваться под о...,01-01


In [6]:
def make_prompt(date, sign):
    return f"<s> день {date} знак {sign} Гороскоп: "

def make_full_prompt(date, sign, text=""):
    return make_prompt(date, sign) + f"{text} <\s>"

In [7]:
from datetime import datetime as dt

make_full_prompt(dt.now().strftime("%m-%d"), "Рыбы")

'<s> день 05-07 знак Рыбы Гороскоп:  <\\s>'

In [8]:
df['features'] = df[['date_fmt', 'sing_ru']].apply(lambda x: make_prompt(x[0], x[1]), axis=1)
df

,sing_ru,date,month,day,weekday,weekday_ru,text,date_fmt,features
0,овен,2004-01-01,1,1,3,Четверг,"Любые разногласия во мнениях скоро улягутся, а...",01-01,<s> день 01-01 знак овен Гороскоп:
1,телец,2004-01-01,1,1,3,Четверг,Первый день нового года прекрасно подходит для...,01-01,<s> день 01-01 знак телец Гороскоп:
2,близнецы,2004-01-01,1,1,3,Четверг,Этот день как нельзя лучше подходит для улучше...,01-01,<s> день 01-01 знак близнецы Гороскоп:
3,рак,2004-01-01,1,1,3,Четверг,"Первый день нового года, да еще понедельник бу...",01-01,<s> день 01-01 знак рак Гороскоп:
4,лев,2004-01-01,1,1,3,Четверг,Львы не сорвутся в начале года с места в карье...,01-01,<s> день 01-01 знак лев Гороскоп:
...,...,...,...,...,...,...,...,...,...
79291,скорпион,2023-01-01,1,1,6,Воскресенье,Сегодня Скорпионов поглотит новогодняя суета –...,01-01,<s> день 01-01 знак скорпион Гороскоп:
79292,стрелец,2023-01-01,1,1,6,Воскресенье,"Сегодня у Стрельцов не все идет по плану, и, в...",01-01,<s> день 01-01 знак стрелец Гороскоп:
79293,козерог,2023-01-01,1,1,6,Воскресенье,Сегодня Козерогам стоит быть во всем осторожне...,01-01,<s> день 01-01 знак козерог Гороскоп:
79294,водолей,2023-01-01,1,1,6,Воскресенье,Сегодня Водолеям придется подстраиваться под о...,01-01,<s> день 01-01 знак водолей Гороскоп:


In [9]:
df['full'] = df[['date_fmt', 'sing_ru', 'text']].apply(lambda x: make_full_prompt(x[0], x[1], x[2]), axis=1)
df

,sing_ru,date,month,day,weekday,weekday_ru,text,date_fmt,features,full
0,овен,2004-01-01,1,1,3,Четверг,"Любые разногласия во мнениях скоро улягутся, а...",01-01,<s> день 01-01 знак овен Гороскоп:,<s> день 01-01 знак овен Гороскоп: Любые разно...
1,телец,2004-01-01,1,1,3,Четверг,Первый день нового года прекрасно подходит для...,01-01,<s> день 01-01 знак телец Гороскоп:,<s> день 01-01 знак телец Гороскоп: Первый ден...
2,близнецы,2004-01-01,1,1,3,Четверг,Этот день как нельзя лучше подходит для улучше...,01-01,<s> день 01-01 знак близнецы Гороскоп:,<s> день 01-01 знак близнецы Гороскоп: Этот де...
3,рак,2004-01-01,1,1,3,Четверг,"Первый день нового года, да еще понедельник бу...",01-01,<s> день 01-01 знак рак Гороскоп:,<s> день 01-01 знак рак Гороскоп: Первый день ...
4,лев,2004-01-01,1,1,3,Четверг,Львы не сорвутся в начале года с места в карье...,01-01,<s> день 01-01 знак лев Гороскоп:,<s> день 01-01 знак лев Гороскоп: Львы не сорв...
...,...,...,...,...,...,...,...,...,...,...
79291,скорпион,2023-01-01,1,1,6,Воскресенье,Сегодня Скорпионов поглотит новогодняя суета –...,01-01,<s> день 01-01 знак скорпион Гороскоп:,<s> день 01-01 знак скорпион Гороскоп: Сегодня...
79292,стрелец,2023-01-01,1,1,6,Воскресенье,"Сегодня у Стрельцов не все идет по плану, и, в...",01-01,<s> день 01-01 знак стрелец Гороскоп:,<s> день 01-01 знак стрелец Гороскоп: Сегодня ...
79293,козерог,2023-01-01,1,1,6,Воскресенье,Сегодня Козерогам стоит быть во всем осторожне...,01-01,<s> день 01-01 знак козерог Гороскоп:,<s> день 01-01 знак козерог Гороскоп: Сегодня ...
79294,водолей,2023-01-01,1,1,6,Воскресенье,Сегодня Водолеям придется подстраиваться под о...,01-01,<s> день 01-01 знак водолей Гороскоп:,<s> день 01-01 знак водолей Гороскоп: Сегодня ...


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, random_state=42, stratify=df['features'] , test_size=0.2)

In [11]:
train_texts = X_train['full']
valid_texts = X_test['full']

In [12]:
with open('train.txt', 'w') as f:
    for t in train_texts:
        f.write(t + '\n')

In [13]:
with open('valid.txt', 'w') as f:
    for t in valid_texts:
        f.write(t + '\n')